In [6]:
# https://www.youtube.com/watch?v=aywZrzNaKjs&ab_channel=Rabbitmetrics

In [2]:
!ollama list

NAME                     	ID          	SIZE  	MODIFIED     
deepseek-r1:8b           	28f8fd6cdc67	4.9 GB	22 hours ago	
llama3.2:1b-instruct-q4_0	53f2745c8077	770 MB	3 months ago	
llama3.2:1b              	baf6a787fdff	1.3 GB	3 months ago	
llama3.1:8b              	42182419e950	4.7 GB	4 months ago	
mistral:instruct         	f974a74358d6	4.1 GB	4 months ago	


In [2]:
import langchain
import langchain_ollama
import langchain_core
import langchain_community
import wikipedia

In [4]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_ollama import OllamaLLM

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


In [41]:
from langchain_core import messages

In [5]:
llm = OllamaLLM(model="deepseek-r1:8b", temperature=0.2)

In [34]:
api_wrapper = WikipediaAPIWrapper(
    top_k_results=3,
    lang="en",
    load_all_available_pages=True,
    load_all_available_meta = True,
    doc_content_chars_max=4000
)
wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)


In [35]:
prompt = PromptTemplate.from_template(
    """Use the following context to answer the question:
    
Context: {context}

Question: {question}

If the information is not in the context, say "I don't have enough information from Wikipedia to answer this."
"""
)

In [47]:
def retrieve_from_wikipedia(question):
    # Wikipedia durchsuchen
    context = wikipedia_tool.run(question)
    
    # Antwort ausgeben
    chain = (
        {"context": lambda x: context, "question": RunnablePassthrough()}
        | prompt 
        | llm 
        | StrOutputParser()
    )
    
    return chain.invoke(question)

In [39]:
from langchain_community.utilities import WikipediaAPIWrapper

def retrieve_from_wikipedia_with_context(question):
    # Konfiguriere API-Wrapper mit mehr Details
    api_wrapper = WikipediaAPIWrapper(
                                        top_k_results=3,
                                        lang="en",
                                        load_all_available_pages=True,
                                        load_all_available_meta = True,
                                        doc_content_chars_max=4000
)
    wikipedia_tool = WikipediaQueryRun(api_wrapper=api_wrapper)
    
    # Wikipedia durchsuchen
    context = wikipedia_tool.run(question)
    
    # Ausführliche Kontextausgabe
    print("===== WIKIPEDIA SEARCH DETAILS =====")
    print(f"Search Query: {question}")
    print(f"Retrieved Context Length: {len(context)} characters")
    print("Context Preview:")
    print(context[:500] + "..." if len(context) > 500 else context)
    print("====================================")
    
    # Antwort generieren
    chain = (
        {"context": lambda x: context, "question": RunnablePassthrough()}
        | prompt 
        | llm 
        | StrOutputParser()
    )
    
    return chain.invoke(question)


In [48]:
# Example usage
result = retrieve_from_wikipedia_with_context("What is DeepSeek?")
print(result)

===== WIKIPEDIA SEARCH DETAILS =====
Search Query: What is DeepSeek?
Retrieved Context Length: 4000 characters
Context Preview:
Page: DeepSeek
Summary: DeepSeek (stylized as deepseek, Chinese: 深度求索; pinyin: Shēndù Qiúsuǒ) is a Chinese artificial intelligence company that develops open-source large language models (LLMs). Based in Hangzhou, Zhejiang, it is owned and funded by Chinese hedge fund High-Flyer, whose co-founder, Liang Wenfeng, established the company in 2023 and serves as its CEO. 
The DeepSeek-R1 model provides responses comparable to other contemporary large language models, such as OpenAI's GPT-4o and o1. I...
<think>
Okay, so I need to figure out what DeepSeek is based on the given context. Let me start by reading through the context carefully.

First, there's a summary for Page: DeepSeek. It mentions that DeepSeek is a Chinese AI company that develops open-source large language models (LLMs). They're based in Hangzhou and owned by High-Flyer Capital, which was founded b